In [15]:
import warnings
warnings.filterwarnings('ignore')
from utils import load_env
load_env()
from crewai import Agent, Task, Crew, LLM
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [16]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [17]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [18]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [19]:
directory_read_tool = DirectoryReadTool(directory='\Crewai Projects\instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [20]:
from crewai.tools import BaseTool

In [21]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [22]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [23]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [24]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [25]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	#memory=True
    max_rpm=6
)

2025-02-04 16:35:09,643 - 4088 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [26]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: I need to gather information about DeepLearningAI, including its company background, key personnel, recent milestones, and potential needs that align with our offerings.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"DeepLearningAI company profile key personnel recent milestones\"}"
## Tool Output: 
{'searchParameters': {'q': 'DeepLearningAI company profile key personnel recent milestones', 'type': 'sear

In [27]:
from IPython.display import Markdown
Markdown(result.raw)

Here are a series of personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng:

---

**Email Draft 1: Integration Focus**

Subject: Enhancing DeepLearningAI's Offerings Through Strategic Integration

Dear Andrew,

I hope this message finds you thriving! I wanted to take a moment to congratulate you on DeepLearningAI’s recent expansion into generative AI and deep learning courses. It’s an exciting time for AI education, and your contributions are paving the way for future innovators.

At CrewAI, we share your commitment to democratizing AI education. We believe that our integration capabilities can significantly enhance your platform’s analytics and workflows, supporting your mission to provide unparalleled access to knowledge globally. 

Let’s discuss how we can partner to further innovate your offerings and amplify the impact of your courses. Would you be open to a quick call next week?

Looking forward to the opportunity!

Best regards,  
[Your Name]  
[Your Position]  
CrewAI

---

**Email Draft 2: Partnership Emphasis**

Subject: Collaboration for Transformative AI Education

Dear Andrew,

Congratulations on the impressive strides DeepLearningAI is making in expanding its educational offerings! Your vision for enhancing accessibility to AI knowledge aligns closely with our mission at CrewAI.

We specialize in creating collaborative solutions that perfectly fit educational frameworks like yours. By exploring a strategic partnership, we can work together to innovate and elevate learning experiences for your global audience.

I’d love to explore this further. Would you be available for a brief discussion next week to brainstorm how our solutions can add value?

Warm regards,  
[Your Name]  
[Your Position]  
CrewAI

---

**Email Draft 3: Innovation Support**

Subject: Supporting DeepLearningAI’s Goals with Innovative Solutions

Dear Andrew,

I hope you are doing well! Watching DeepLearningAI’s journey toward democratizing AI education has been incredibly inspiring, particularly your recent focus on generative AI.

At CrewAI, we understand the challenges that come with continuous innovation. Our solutions are designed to support organizations like yours by enhancing educational efficiency and creating immersive learning experiences.

Could we schedule a meeting to discuss how our offerings might align with your current initiatives and drive future success? 

Thank you for considering this opportunity, and I look forward to connecting!

Best,  
[Your Name]  
[Your Position]  
CrewAI

---

These drafts are crafted to resonate with Andrew Ng’s objectives while positioning CrewAI as a valuable partner in DeepLearningAI's mission.